# PyTorch model with 5 modes (for now)
*Step 1:* Collect and label data</br>
*Step 2:* process data and normalize it!</br>
*Step 3:* Convert text to numeric vectors</br>
*Step 4:* Train a Lightweight Classifier</br>
*Step 5:* Evaluate Performance</br>
*Step 6:* Save and Load model to the project</br>

### SQL_mode, insight_mode, Comparison_mode, visualization_mode, prediction_mode


In [181]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sentence_transformers import SentenceTransformer
from torch.utils.data import TensorDataset, DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F
import json
from pathlib import Path


In [182]:
### Prepare the data and join into one df
comparison_df = pd.read_csv("training_data_modes/comparison_mode.csv")
insight_df = pd.read_csv("training_data_modes/insight_mode.csv")
prediction_df = pd.read_csv("training_data_modes/prediction_mode.csv")
sql_df = pd.read_csv("training_data_modes/sql_mode.csv")
visualization_df = pd.read_csv("training_data_modes/visualization_mode.csv")



df = pd.concat([comparison_df, insight_df, prediction_df, sql_df, visualization_df])
X, y = df['text'], df['label']

label_to_int = {
  "sql_mode": 0,
  "insight_mode": 1,
  "comparison_mode": 2,
  "visualization_mode": 3,
  "prediction_mode": 4
}

for _, row in df.iterrows():
    row['label'] = label_to_int[row['label']]
df.head()

,text,label
0,Which period had better transactions?,2
1,Which period had better expenses?,2
2,How does conversion rate differ between regions?,2
3,How does production costs differ between regions?,2
4,Compare customers from this quarter to last qu...,2


In [183]:
#### Shuffle the data and split the train test.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = y_train.tolist()
y_test = y_test.tolist()

In [184]:
### Get the embedding model to translate text to vector
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encode train/test text sets
X_train_embeddings = model.encode(X_train)
X_test_embeddings = model.encode(X_test)

# Convert embeddings to tensors
X_train_tensor = torch.tensor(X_train_embeddings, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_embeddings, dtype=torch.float32)

# Convert labels to tensors (important!)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [185]:
### Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [186]:
class MLPClassifier(nn.Module):
    def __init__(self, input_dim=384, hidden1=128, hidden2=64, output_dim=5):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden1)
        self.fc2 = nn.Linear(hidden1, hidden2)
        self.fc3 = nn.Linear(hidden2, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)          # final layer → logits
        return x

In [187]:
### Fit the model
model = MLPClassifier(input_dim=384, output_dim=5)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [208]:
### Optimize the NN
for embeddings, labels in train_loader:
    optimizer.zero_grad()
    outputs = model(embeddings)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()

In [209]:
### Evaluate NN
import torch
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np

# ✅ 1️⃣ Switch model to evaluation mode
model.eval()

# ✅ 2️⃣ Disable gradient calculations
all_preds = []
all_labels = []

with torch.no_grad():
    for embeddings, labels in test_loader:
        # Forward pass
        outputs = model(embeddings)

        # Get predicted class (highest logit)
        _, preds = torch.max(outputs, 1)

        # Store results for later metrics
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# ✅ 3️⃣ Compute accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"✅ Test Accuracy: {accuracy:.4f}")

# ✅ 4️⃣ Detailed performance report
print("\n📊 Classification Report:")
print(classification_report(all_labels, all_preds, digits=3))



✅ Test Accuracy: 0.7900

📊 Classification Report:
              precision    recall  f1-score   support

           0      0.000     0.000     0.000        38
           1      0.662     0.979     0.790        48
           2      0.914     0.970     0.941        33
           3      0.913     0.955     0.933        44
           4      0.771     1.000     0.871        37

    accuracy                          0.790       200
   macro avg      0.652     0.781     0.707       200
weighted avg      0.653     0.790     0.711       200



/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [190]:
# Create a models folder if it doesn't exist
Path("models").mkdir(exist_ok=True)

# Save the model weights
torch.save(model.state_dict(), "models/mlp_mode_classifier.pt")

print("✅ Model saved successfully: models/mlp_mode_classifier.pt")

✅ Model saved successfully: models/mlp_mode_classifier.pt


In [191]:
label_map = {
    0: "sql_mode",
    1: "insight_mode",
    2: "comparison_mode",
    3: "visualization_mode",
    4: "prediction_mode"
}

with open("models/label_map.json", "w") as f:
    json.dump(label_map, f)

✅ Label map saved successfully: models/label_map.json


In [192]:
# Load label map
with open("models/label_map.json", "r") as f:
    label_map = json.load(f)

# Initialize model
model = MLPClassifier(input_dim=384, hidden1=128, hidden2=64, output_dim=len(label_map))
model.load_state_dict(torch.load("models/mlp_mode_classifier.pt", map_location=torch.device("cpu")))
model.eval()

✅ Model and label map loaded successfully!
